In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import itertools

t1 = np.random.rand(3,3)
t2 = np.random.rand(3,3)

In [ ]:
# 順列の符号函数函数 sgn を転倒数によって定義する

def sgn(lst):
  count = 0
  n = len(lst)
  for i in range(n):
    for j in range(i):
      if lst[j] > lst[i]:
        count += 1
  return (-1)**count


lst = np.array([2, 0, 1, 3, 4])
sgn(lst)

1

In [ ]:
# テンソル の交代化射影 alt を定義する

def alt(tensor):
  k = len(np.shape(tensor)) # テンソルの階数
  n = np.shape(tensor)[0] # ベクトルの次元
  res = np.zeros(np.shape(tensor)) #結果を格納する配列を初期化
  # 添字の集合0,1,...,(n-1)から順序区別して区別してkを取る
  for idx in itertools.permutations(np.arange(n), k):
    for pmt in itertools.permutations(idx, len(idx)):
      res[idx] += sgn(idx)*sgn(pmt)*tensor[pmt]
  return (1/math.factorial(k))*res

a1 = alt(t1)
a2 = alt(t2)

In [ ]:
# テンソル代数同士の内積 dot_t, 交代代数同士の内積 dot_e を定義する

def dot_t(tensor1, tensor2):
  return (tensor1*tensor2).sum()

def dot_e(alttensor1, alttensor2):
  return (1/math.factorial(len(np.shape(alttensor1)))
  * dot_t(alttensor1, alttensor2))

print(dot_t(t1, t2))
print(dot_e(a1, a2))

1.2865562395703343
0.01136322805380792


In [ ]:
# k個の列ベクトルを並べた並べた(n, k)行列をk階テンソルにする関数 tensorize を定義する

# こっちはどこかで間違えた
def tens(mat):
  dim, rank = np.shape(mat)
  lst = [dim]*rank
  res = np.zeros(lst)
  for idx in itertools.combinations_with_replacement(np.arange(dim), rank):
    res[idx] = mat[[idx],[np.arange(rank)]].prod()
  return res

def tensorize(mat):
  dim, rank = np.shape(mat)
  if rank == 2:
    submat = mat[:, (rank-2)]
  else:
    submat = mat[:, 0:(rank-1)]
  return np.tensordot(submat, mat[:, rank-1], axes=0)

mat = np.array([[1,2], [0,1], [2,2]])
print(tensorize(mat))


[[2 1 2]
 [0 0 0]
 [4 2 4]]


In [ ]:
# comass

def comass(anttensor):
  k = len(np.shape(anttensor))
  n = np.shape(anttensor)[0]
  lst = np.zeros(100000)
  for i in range(100000):
    lst[i] = dot_t(anttensor, simple_vectors[i,:,:])
  return np.amax(abs(lst))

# 1になるはずなのだが、、、
comass(2*alt(tensorize(np.array([[0,0], [1,0.1], [0,1], [0,0.3]]))))

1.017135409280659

In [ ]:
# tensor product

a = np.array([[1,0], [2, 1]])
b = np.array([[2, 1], [1, 0], [1, 9]])

c = np.tensordot(a,b, axes=0)
c.shape

(2, 2, 3, 2)

In [ ]:
# exterior product

def exteroir_product(alttensor1, alttensor2):
  return 1/math.factorial(len(alttensor1.shape))*1/math.factorial(len(alttensor2.shape))*alt(
      np.tensordot(alttensor1, alttensor2, axes=0)
  )



In [ ]:
# simple vectors
"""
times = 10000000
n = 4
k = 2
simple_vectors = np.zeros((times, n, n))
matrices = np.random.rand(times*n*k).reshape(times, n, k)
for i in range(times):
  tensor = tensorize(matrices[i,:,:])
  simple_vectors[i,:,:] = tensor/np.sqrt(dot_t(tensor, tensor))
"""



'\ntimes = 10000000\nn = 4\nk = 2\nsimple_vectors = np.zeros((times, n, n))\nmatrices = np.random.rand(times*n*k).reshape(times, n, k)\nfor i in range(times):\n  tensor = tensorize(matrices[i,:,:])\n  simple_vectors[i,:,:] = tensor/np.sqrt(dot_t(tensor, tensor))\n'

In [ ]:
simple_vectors

np.save('/content/sample_data/simple_vectors', simple_vectors)